<a href="https://colab.research.google.com/github/haeniKim/metaverse-academy/blob/master/DL/230628_pytorch_boston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pytorch를 활용한 Boston House 문제 - 회귀

* linear
* x, y

In [ ]:
from sklearn.datasets import fetch_openml

dataset = fetch_openml(name='boston')

dataset.keys()

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset['data'])
df.columns = dataset['feature_names']
df['target'] = dataset['target']

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [ ]:
#입력값 13, 최종 출력 1
#신경망 여러개를 쌓고 싶으면 output을 여러개 잡기.

import torch
import torch.nn as nn
from torch.optim.adam import Adam


In [ ]:
df['CHAS'] = df['CHAS'].astype('float')
df["RAD"] = df["RAD"].astype('float')

In [ ]:
model = nn.Sequential(
    nn.Linear(13, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
)

In [ ]:
X_train = df.iloc[:, :13].values
X_train

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [ ]:
y_train = df.iloc[:, -1].values
y_train.shape

(506,)

In [ ]:
#배치 단위 학습
batch_size = 100
optim = Adam(model.parameters(), lr = 0.001)
epochs = 200

for epoch in range(epochs):
  for temp in range(len(X_train)//batch_size):
    s = temp * batch_size
    e = s + batch_size

    # 항상 tensor로 넣어야 함.
    x = torch.FloatTensor(X_train[s:e])
    y = torch.FloatTensor(y_train[s:e])

    optim.zero_grad()
    pred = model(x)
    loss = nn.MSELoss()(pred, y)
    loss.backward()
    optim.step() # 다음 단계로 넘어감

  if epoch % 10 == 0 :
    print(f'{epoch+1} loss : {loss.item()}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


1 loss : 73.00157928466797
11 loss : 44.92757797241211
21 loss : 42.26953125
31 loss : 40.97299575805664
41 loss : 40.37531661987305
51 loss : 40.00135803222656
61 loss : 39.209510803222656
71 loss : 38.6927490234375
81 loss : 38.39250564575195
91 loss : 38.17924880981445
101 loss : 37.87012481689453
111 loss : 37.714107513427734
121 loss : 37.62895584106445
131 loss : 37.578609466552734
141 loss : 37.52055740356445
151 loss : 37.46042251586914
161 loss : 37.42578125
171 loss : 37.43179702758789
181 loss : 37.45205307006836
191 loss : 37.44827651977539


In [ ]:
X_train[0, : 13]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [ ]:
pred = model(torch.FloatTensor(X_train[0, : 13]))
pred

tensor([26.1489], grad_fn=<AddBackward0>)

In [ ]:
y_train[0]

24.0

In [ ]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.1367, -0.0789, -0.0822,  ..., -0.2145,  0.0550,  0.2541],
                      [-0.0104, -0.1296,  0.1300,  ...,  0.2499, -0.2066,  0.2833],
                      [ 0.0881, -0.0805,  0.1160,  ..., -0.0650, -0.1353,  0.0702],
                      ...,
                      [ 0.0869, -0.0216, -0.1496,  ...,  0.1193, -0.2062, -0.0981],
                      [ 0.0350,  0.0438, -0.1006,  ...,  0.0702, -0.0210, -0.1002],
                      [-0.2442,  0.0625, -0.1054,  ...,  0.2092, -0.0260,  0.1974]])),
             ('0.bias',
              tensor([ 0.2519, -0.1613, -0.1690, -0.1722,  0.0601, -0.2015, -0.0647, -0.2242,
                       0.1067,  0.1076,  0.1919, -0.2041, -0.0937,  0.2517, -0.0703, -0.1912,
                       0.1621, -0.1801, -0.0511, -0.1330,  0.1823, -0.2755, -0.1245, -0.1240,
                      -0.0330, -0.1859, -0.2749, -0.1335,  0.0559, -0.2826,  0.2275,  0.1420,
                       0.3011,  0.0391, 

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
model_new = model()
model_new.load_state_dict(torch.load('model.pth'))